<a href="https://colab.research.google.com/github/PhilippMatthes/diplom/blob/master/src/shl-deep-learning-timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a deep CNN to directly classify SHL timeseries data

In [1]:
# Get needed auxiliary files for colab
!git clone https://github.com/philippmatthes/diplom
%cd /content/diplom/src
!mkdir shl-dataset
!wget -nc -O shl-dataset/challenge-2019-train_torso.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
!wget -nc -O shl-dataset/challenge-2019-train_bag.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
!wget -nc -O shl-dataset/challenge-2019-train_hips.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_hips.zip
!wget -nc -O shl-dataset/challenge-2020-train_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-train_hand.zip

Cloning into 'diplom'...
remote: Enumerating objects: 1375, done.
remote: Counting objects: 100% (712/712), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 1375 (delta 344), reused 582 (delta 229), pack-reused 663
Receiving objects: 100% (1375/1375), 27.92 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (720/720), done.
/content/diplom/src
--2021-08-07 09:10:53--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-train_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  9.25MB/s    in 9m 48s  

2021-08-07 09:20:41 (9.49 MB/s) - ‘shl-dataset/challenge-2019-train_torso.zip’ saved [5852446972/5852446972]

--2021-08-07 09:20:4

In [1]:
# Switch to src dir and select tensorflow
%cd /content/diplom/src
%tensorflow_version 2.x

/content/diplom/src


In [2]:
# Define all datasets to train our model on

from pathlib import Path

DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-train_torso.zip'),
    Path('shl-dataset/challenge-2019-train_bag.zip'),
    Path('shl-dataset/challenge-2019-train_hips.zip'),
    Path('shl-dataset/challenge-2020-train_hand.zip'),
]

In [3]:
# Create scalers that we will fit on our data

from sklearn.preprocessing import PowerTransformer

acc_scaler = PowerTransformer()
mag_scaler = PowerTransformer()
gyr_scaler = PowerTransformer()

In [4]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6738430798524105472, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16183459840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14111909831258419595
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [5]:
import numpy as np

from tools.dataset import load_zipped_shl_dataset
from tools.export import export_power_transformer

from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Join all datasets
acc_mag_conc = None
mag_mag_conc = None
gyr_mag_conc = None
y_conc = None

for dataset_dir in DATASET_DIRS:
    # Load dataset from zip file into temporary directory
    dataset = load_zipped_shl_dataset(dataset_dir, tqdm=tqdm)
    if acc_mag_conc is None:
        acc_mag_conc = dataset.acc_mag
    else:
        acc_mag_conc = np.concatenate((acc_mag_conc, dataset.acc_mag), axis=0)
    if mag_mag_conc is None:
        mag_mag_conc = dataset.mag_mag
    else:
        mag_mag_conc = np.concatenate((mag_mag_conc, dataset.mag_mag), axis=0)
    if gyr_mag_conc is None:
        gyr_mag_conc = dataset.gyr_mag
    else:    
        gyr_mag_conc = np.concatenate((gyr_mag_conc, dataset.gyr_mag), axis=0)
    if y_conc is None:
        y_conc = dataset.labels
    else:
        y_conc = np.concatenate((y_conc, dataset.labels), axis=0)
    del dataset
    

Extracting shl-dataset/challenge-2019-train_torso.zip: 100%|██████████| 22/22 [03:42<00:00, 10.10s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done


Extracting shl-dataset/challenge-2019-train_bag.zip: 100%|██████████| 22/22 [03:47<00:00, 10.32s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done


Extracting shl-dataset/challenge-2019-train_hips.zip: 100%|██████████| 22/22 [03:47<00:00, 10.34s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done


Extracting shl-dataset/challenge-2020-train_hand.zip: 100%|██████████| 23/23 [03:46<00:00,  9.83s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done


In [6]:
# Check that we don't have NaNs in our dataset
assert not np.isnan(acc_mag_conc).any()
assert not np.isnan(mag_mag_conc).any()
assert not np.isnan(gyr_mag_conc).any()

# Fit and export scalers
acc_mag_scaled = acc_scaler.fit_transform(acc_mag_conc)
del acc_mag_conc
mag_mag_scaled = mag_scaler.fit_transform(mag_mag_conc)
del mag_mag_conc
gyr_mag_scaled = gyr_scaler.fit_transform(gyr_mag_conc)
del gyr_mag_conc

export_power_transformer(acc_scaler, 'models/acc-transformer.json')
export_power_transformer(mag_scaler, 'models/mag-transformer.json')
export_power_transformer(gyr_scaler, 'models/gyr-transformer.json')

# Prepare training data
X = np.stack([
    acc_mag_scaled,
    mag_mag_scaled, 
    gyr_mag_scaled,
], axis=2)
del acc_mag_scaled
del mag_mag_scaled
del gyr_mag_scaled

In [7]:
# Install imblearn, a package with functionalities to balance our dataset
import sys
!{sys.executable} -m pip install imbalanced-learn

In [ ]:
# Balance our dataset using the SMOTE method
from imblearn.over_sampling import SMOTE

balancer = SMOTE()

# To apply SMOTE, we have to temporarily reshape 
# X from 3d (n_samples, n_timesteps, n_sensors) 
# into 2d (n_samples, n_timesteps * n_sensors) 
# and back after balancing
X_balanced, y_balanced = balancer.fit_resample(
    X.reshape((X.shape[0], X.shape[1] * X.shape[2])), y_conc
)
X_balanced = X_balanced.reshape(
    X_balanced.shape[0], X.shape[1], X.shape[2]
)

del X
del y_conc

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Check that our data was balanced successfully
from sklearn.utils.class_weight import compute_class_weight
print(compute_class_weight('balanced', classes=np.unique(y_balanced), y=y_balanced))
print(y_balanced.shape, X_balanced.shape)

In [ ]:
# Create our model

from tensorflow.keras import layers

from architectures.baseline_cnn import make_baseline_cnn
from tools.dataset import shl_dataset_label_order

MODEL_DIR = Path('models/shl-baseline-cnn.h5')
OVERRIDE_MODEL = True

if MODEL_DIR.is_file() and not OVERRIDE_MODEL:
    # Load model for transfer learning
    model = keras.models.load_model(MODEL_DIR)
else:
    model = make_baseline_cnn(
        input_shape=[X_balanced.shape[1], X_balanced.shape[2]], 
        output_classes=len(shl_dataset_label_order)
    )

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['acc']
    )

In [ ]:
model.summary()

In [ ]:
# Train model
callbacks = [
    keras.callbacks.CSVLogger(f'{MODEL_DIR}.log', append=MODEL_DIR.is_file()),
    keras.callbacks.ModelCheckpoint(
        str(MODEL_DIR), save_best_only=True, monitor='val_loss', verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1),
]
model.fit(
    X_balanced, 
    y_balanced,
    epochs=200, 
    batch_size=32,
    callbacks=callbacks,
    validation_split=0.1,
    verbose=1,
    shuffle=True
)